# Libraries

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os
import time
dotenv_path = r'C:\Users\santi\OneDrive\Desktop\Spotify_Popularity\.env.txt'
load_dotenv(dotenv_path)

# Api Connection

In [ ]:
# Configura tus credenciales de cliente
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# Autenticación con Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [ ]:
from minim import spotify
import pandas as pd
client = spotify.WebAPI(flow="pkce", browser=True, web_framework="http.server",
                        scopes=spotify.WebAPI.get_scopes("all"), overwrite=True)

# Specify the target Spotify playlist's ID.
playlist_id = "bff27fb8fb714104"

# Get the first 100 playlist items/tracks.
resp = client.get_playlist_items(playlist_id, limit=100)

# Get the track IDs of the returned playlist items/tracks.
track_ids = [item["track"]["id"] for item in resp["items"]]

# Get the audio features for the 100 tracks.
afs = client.get_tracks_audio_features(track_ids)

# Create a DataFrame using the audio features data.
df = pd.DataFrame(afs)

# Iterate through the rest of the playlist items/tracks.
while True:

    # Break the loop if all playlist items/tracks have been retrieved.
    if resp["offset"] + resp["limit"] >= resp["total"]:
        break

    # Same procedure as above.
    resp = client.get_playlist_items(playlist_id, limit=100, 
                                     offset=resp["offset"] + resp["limit"])
    track_ids = [item["track"]["id"] for item in resp["items"]]
    afs = client.get_tracks_audio_features(track_ids)

    # Concatenate the new audio features data to the existing DataFrame.
    df = pd.concat([df, pd.DataFrame(afs)])

# Dataset

In [ ]:
%%time
# Obtener el ID de la playlist
dfs = []
#playlist_id = ['1M31YfWCBmOSlGGyF0wvsw', '37i9dQZF1DX4KeocBrdbJg', '37i9dQZEVXbMDoHDwVN2tF', '37i9dQZF1DXbvPjXfc8G9S',
#              '4KlHX1H41ZIyQ5oodAp2kj', '7xwYHh7KbKDtlMblSkvtfu']

playlist_id = ['1M31YfWCBmOSlGGyF0wvsw']

for pl in playlist_id:
    # Obtener las canciones de la playlist
    results = sp.playlist_tracks(pl)
    
    # Listas para almacenar los datos
    song_names = []
    song_ids = []
    audio_features = []
    popularity = []
    artists = []
    release_years = []
    
    # Iterar sobre las canciones y obtener los datos
    for track in results['items']:
        # Obtener datos de la canción
        song_name = track['track']['name']
        song_id = track['track']['id']
        song_names.append(song_name)
        song_ids.append(song_id)
    
        # Obtener características de audio
        features = sp.audio_features([song_id])[0]
        audio_features.append(features)
    
        # Obtener popularidad de la canción
        song_popularity = track['track']['popularity']
        popularity.append(song_popularity)
    
        # Obtener género de la canción
        artist_info = track['track']['artists'][0]  # Obtener la información del primer artista de la canción
        artist_name = artist_info['name']
        artists.append(artist_name)
    
        # Obtener año de lanzamiento de la canción
        track_info = sp.track(song_id)
        release_date = track_info['album']['release_date']
        release_year = release_date.split('-')[0] if release_date else None
        release_years.append(release_year)
    
   
    # Crear DataFrame con los datos
    df = pd.DataFrame({
        'Song ID': song_ids,
        'Song Name': song_names,
        'Artist Name': artists,
        'Year': release_years,
        'Popularity': popularity
    })
    
    df_audio_features = pd.DataFrame(audio_features)
    df_completo = pd.merge(df, df_audio_features, how='left', left_on='Song ID', right_on='id')
    df_completo = df_completo.drop(columns=['type', 'id', 'uri', 'track_href', 'analysis_url'], axis=1)
    dfs.append(df_completo)

tablon = pd.concat(dfs)
tablon = tablon.drop_duplicates(subset='Song ID').reset_index(drop=True)